In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "f_cogs")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.SAP_VCDSDBL02"
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW temp_f_cogs
    AS
    SELECT
        farm_fingerprint(CONCAT(
            IFNULL(CAST(calday AS STRING), ""), 
            IFNULL(CAST(plant AS STRING), ""), 
            IFNULL(CAST(material AS STRING), "")
        )) hash_id,
        calday,
        plant store_id,
        material product_id,
        SUM(CASE WHEN distr_chan IN ('10', '20') THEN cost ELSE 0 END) cost,
        SUM(CASE WHEN distr_chan IN ('10', '20') THEN quant_b ELSE 0 END) quantity,
        IFNULL(
            SUM(CASE WHEN distr_chan IN ('10', '20') THEN cost ELSE 0 END) / 
            NULLIF(SUM(CASE WHEN distr_chan IN ('10', '20') THEN quant_b ELSE 0 END), 0),
        0) cogs_per_unit,
        SUM(cost) cost_b2b,
        SUM(quant_b) quantity_b2b,
        IFNULL(CASE WHEN SUM(quant_b) = 0 THEN 0 ELSE SUM(cost) / SUM(quant_b) END, 0) cogs_per_unit_b2b
    FROM (
        SELECT calday, plant, material, distr_chan,
            CASE 
                WHEN cost < 0.000001 AND cost > 0 THEN 0
                WHEN cost > -0.000001 AND cost < 0 THEN 0
                ELSE cost
            END cost,
            CASE 
                WHEN quant_b < 0.000001 AND quant_b > 0 THEN 0
                WHEN quant_b > -0.000001 AND quant_b < 0 THEN 0
                ELSE quant_b
            END quant_b 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdsdbl02 a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_customer_master dcm ON a.sold_to = dcm.id
        WHERE calday IN ({CALDAY_IN_STR})
    ) a 
    GROUP BY
        calday,
        plant,
        material;
    """)

else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != '':
    # Get the calday list string from temp table
    calday_query = """
    SELECT
        CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
    FROM temp_f_cogs
    """

    calday_df = spark.sql(calday_query)
    calday_in_str = calday_df.collect()[0][0]

    # Only run DELETE if the string is non-empty
    if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
        spark.sql(f"""
        DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs
        WHERE calday IN ({calday_in_str})
        """)
    else:
        print("No valid CALDAYs found to delete.")
        
else:
    print(f"No valid calday's found In common-etl to create the view: temp_{target_table}")

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    ).display()
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dwh.f_cogs can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")